In [2]:
from dbase.DataAPI.ThetaData import * #type: ignore
from dbase.database.SQLHelpers import * #type: ignore
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from EventDriven.eventScheduler import *
from trade.backtester_.backtester_ import PTDataset, PTBacktester
import pandas_ta as ta
from trade.assets.Stock import Stock
from trade.backtester_.utils.WalkForwardUtils import prev_monday 
from trade.backtester_.strats import MAStrat
import yfinance as yf
from datetime import datetime
from EventDriven.backtest import OptionSignalBacktest
%load_ext autoreload
%autoreload 2
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)


Console Logging & File Logging Can be configured using STREAM_LOG_LEVEL and FILE_LOG_LEVEL in environment variables.
Propagate to root logger can be set using PROPAGATE_TO_ROOT_LOGGER in environment variables.
Example:
STREAM_LOG_LEVEL = 'DEBUG'
FILE_LOG_LEVEL = 'INFO'
PROPAGATE_TO_ROOT_LOGGER = 'False'

2025-02-24 20:42:56 trade.helpers.Logging INFO: Logging Root Directory: c:\Users\Zino\python-playground\QuantTools\logs
Using Proxy URL: http://18.232.166.224:5500/thetadata


In [3]:

def create_datasate(stocks: list, start: str,interval: str, engine: str = 'yf', timewidth = None, timeframe = None, end: str = datetime.today(), return_object = False ):
    dataset = []
    if engine.lower() == 'yf':
        for stock in stocks:
            start = prev_monday(start)
            data2 = yf.download(stock, start = start, end = end, interval=interval, progress = False)

            dataset.append(PTDataset(stock, data2))
    else:
        for stk in stocks:
            stock = Stock(stk)
            data = stock.spot(ts = True, ts_start = '2018-01-01')
            data.rename(columns = {x:x.capitalize() for x in data.columns}, inplace= True)
            data['Timestamp'] = pd.to_datetime(data['Timestamp'], format = '%Y-%m-%d')
            data2 = data.set_index('Timestamp')
            data2 = data2.asfreq('W', method = 'ffill')
            data2 = data2.fillna(0)
            data2['Next_Day_Open'] = data2.Open.shift(-1)
            data2['EMA'] = ta.ma('ema', data2.Close, length = 21).fillna(0)
            dataset.append(PTDataset(stk, data2))
    return dataset if return_object else data2

In [4]:
start, end, interval = '2023-05-29', '2024-05-28','1d'
STOCKS = ['AAPL', 'MSFT','GOOGL', 'AMD', 'AMZN']
dataset = create_datasate(STOCKS, start, interval,end = end , return_object=True)
MAStrat.start_date = pd.to_datetime('1994-03-22')
tt = PTBacktester(dataset, MAStrat, cash =1000, commission = 0.0035)
stats = tt.run()
trades = tt.trades()
shorts = tt.trades()[tt.trades()['Size'] < 0]

YF.download() has changed argument auto_adjust default to True


In [5]:
trades_ = trades[:20]
trades_

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Tag,Duration,Ticker
0,-8,37,38,120.796297,120.921649,-1.002816,-0.001038,2023-07-24,2023-07-25,None,1 days,GOOGL
0,-9,37,38,110.232833,111.139999,-8.164500,-0.008230,2023-07-24,2023-07-25,None,1 days,AMD
1,7,39,40,130.053799,131.194407,7.984252,0.008770,2023-07-26,2023-07-27,None,1 days,GOOGL
1,-8,39,40,111.518319,111.790001,-2.173458,-0.002436,2023-07-26,2023-07-27,None,1 days,AMD
0,-2,40,41,335.377808,329.824221,11.107174,0.016559,2023-07-27,2023-07-28,None,1 days,MSFT
2,-8,41,42,112.983167,114.160004,-9.414691,-0.010416,2023-07-28,2023-07-31,None,3 days,AMD
0,7,42,43,133.666197,133.550003,-0.813357,-0.000869,2023-07-31,2023-08-01,None,1 days,AMZN
1,-3,43,44,330.167057,329.784663,1.147182,0.001158,2023-08-01,2023-08-02,None,1 days,MSFT
3,-8,43,44,113.860092,119.489998,-45.039246,-0.049446,2023-08-01,2023-08-02,None,1 days,AMD
2,-3,45,46,321.114774,328.054887,-20.820341,-0.021613,2023-08-03,2023-08-04,None,1 days,MSFT


In [6]:
import json
with open('../input/profitable_weights.json', 'r') as f:
    weights = json.load(f)
trades_ = pd.read_csv('../input/profitable_trades.csv').iloc[:, 1:]

w_map = {x: w  * 0.75 for x, w in weights.items()}
w_map

{'TSLA': 0.13065599891579077,
 'NVDA': 0.10272520137767008,
 'AAPL': 0.07517530231362687,
 'MSFT': 0.07850475106329509,
 'AVGO': 0.05774176540220492,
 'SBUX': 0.059315194762236595,
 'AMD': 0.022715412757568706,
 'GOOG': 0.04694712470612189,
 'BAC': 0.04260397697944098,
 'QCOM': 0.018892840415863976,
 'AMZN': 0.020683590911291364,
 'HD': 0.0267984089541643,
 'BA': 0.016485862139615462,
 'DIS': 0.018495769796427215,
 'MU': 0.007893535918856541,
 'PFE': 0.013115263585825373,
 'WMT': 0.00375,
 'JNJ': 0.00375,
 'INTC': 0.00375}

In [7]:
trades_ = trades_[:5]

In [27]:
#Backtest class 
evb_backtest = OptionSignalBacktest(trades_)

In [28]:
evb_backtest.portfolio.weight_map = w_map
evb_backtest.portfolio.weight_map
evb_backtest.portfolio.order_settings = {'type': 'naked',
 'specifics': [{'direction': 'long',
   'rel_strike': .900,
   'dte': 365,
   'moneyness_width': 0.15},
   {'direction': 'short',
   'rel_strike': .80,
   'dte': 365,
   'moneyness_width': 0.15}],

 'name': 'vertical_spread'}
evb_backtest.portfolio.max_price = int(10)

Symbol NVDA not being processed but present in weight_map
Symbol SBUX not being processed but present in weight_map
Symbol AMD not being processed but present in weight_map
Symbol GOOG not being processed but present in weight_map
Symbol BAC not being processed but present in weight_map
Symbol QCOM not being processed but present in weight_map
Symbol HD not being processed but present in weight_map
Symbol BA not being processed but present in weight_map
Symbol DIS not being processed but present in weight_map
Symbol MU not being processed but present in weight_map
Symbol PFE not being processed but present in weight_map
Symbol WMT not being processed but present in weight_map
Symbol JNJ not being processed but present in weight_map
Symbol INTC not being processed but present in weight_map


In [10]:
signals = evb_backtest.bars.signal_df
signals

,Date,TSLA,AAPL,MSFT,AVGO,AMZN
0,2023-07-05,1,1,1,1,1
1,2023-07-06,0,0,0,0,0
2,2023-07-07,0,0,0,0,0
3,2023-07-08,0,0,0,0,0
4,2023-07-09,0,0,0,0,0
5,2023-07-10,0,0,0,0,0
6,2023-07-11,0,0,0,0,0
7,2023-07-12,0,0,0,0,0
8,2023-07-13,0,0,0,0,0
9,2023-07-14,0,0,0,0,0


In [29]:
import cProfile
import pstats
import io

profiler = cProfile.Profile()
profiler.enable()
#run backtest

evb_backtest.run()
profiler.disable()
stream = io.StringIO()
stats = pstats.Stats(profiler, stream=stream).sort_stats('cumulative')

        Date  TSLA  AAPL  MSFT  AVGO  AMZN
0 2023-07-05     1     1     1     1     1
Processing event: MARKET
Processing event: SIGNAL
Processing event: SIGNAL
Processing event: SIGNAL
Processing event: SIGNAL
Processing event: SIGNAL
Processing event: ORDER
Processing event: ORDER
Processing event: ORDER
Processing event: ORDER
Processing event: FILL
Processing event: FILL
Processing event: FILL
Processing event: FILL
Event queue is empty, processed 14 event(s)
        Date  TSLA  AAPL  MSFT  AVGO  AMZN
1 2023-07-06     0     0     0     0     0
Processing event: MARKET
Event queue is empty, processed 1 event(s)
        Date  TSLA  AAPL  MSFT  AVGO  AMZN
2 2023-07-07     0     0     0     0     0
Processing event: MARKET
Event queue is empty, processed 1 event(s)
        Date  TSLA  AAPL  MSFT  AVGO  AMZN
3 2023-07-08     0     0     0     0     0
Processing event: MARKET
Event queue is empty, processed 1 event(s)
        Date  TSLA  AAPL  MSFT  AVGO  AMZN
4 2023-07-09     0     0   

In [46]:
evb_backtest.portfolio.plot_portfolio()

In [ ]:
trades_df = evb_backtest.portfolio.get_trades().head(5)
# trades_df['TotalEntryCost'] = trades_df['EntryPrice'] * trades_df['Quantity'] + trades_df['EntryCommission'] + trades_df['EntrySlippage']
# trades_df['AuxilaryEntryCost'] = trades_df['EntryMarketValue'] - trades_df['TotalEntryCost'] 
# trades_df['TotalExitCost'] = trades_df['ExitPrice'] * trades_df['Quantity'] + trades_df['ExitCommission'] + trades_df['ExitSlippage']
# trades_df['AuxilaryExitCost'] = trades_df['ExitMarketValue'] - trades_df['TotalExitCost'] 
trades_df

,Ticker,PnL,ReturnPct,EntryPrice,EntryCommission,EntrySlippage,EntryMarketValue,ExitPrice,ExitCommission,ExitSlippage,ExitMarketValue,Quantity,EntryTime,ExitTime,Duration,Positions,TotalEntryCost,AuxilaryEntryCost,TotalExitCost,AuxilaryExitCost
0,TSLA,-106.606105,-21.449018,497.020901,6.417,-17.436275,11425.063725,390.414797,6.417,15.957321,8985.957321,23,2023-07-05,2023-08-02,28,&L:TSLA20240621C330&S:TSLA20240621C346.67,11420.461450,4.602275,9001.914642,-15.957321
1,AAPL,35.040598,7.146842,490.294866,3.627,6.706259,6370.206259,525.335464,3.627,-5.011965,6832.988035,13,2023-07-05,2023-08-04,30,&L:AAPL20240621C230&S:AAPL20240621C280,6384.166519,-13.960259,6827.976070,5.011965
2,MSFT,-103.429300,-21.284534,485.936416,3.906,9.203818,6799.203818,382.507116,3.906,4.005622,5359.005622,14,2023-07-05,2023-08-09,35,&L:MSFT20240621C355&S:MSFT20240621C365,6816.219635,-17.015818,5363.011244,-4.005622
3,AMZN,-53.537275,-10.793833,495.998748,0.837,2.159245,1487.159245,442.461473,0.837,0.721420,1328.221420,3,2023-07-05,2023-10-25,112,&L:AMZN20240621C145&S:AMZN20240621C160,1490.992490,-3.833245,1328.942840,-0.721420


In [48]:

weighted_holdings_df = pd.DataFrame(evb_backtest.portfolio.weighted_holdings).set_index('datetime')
head = weighted_holdings_df.head()
head['cash_total'] = head.drop(columns=['cash','commission', 'total']).sum(axis = 1)
head

,TSLA,AAPL,MSFT,AVGO,AMZN,NVDA,JNJ,HD,BA,WMT,INTC,QCOM,SBUX,AMD,MU,DIS,BAC,GOOG,cash,commission,total,cash_total
datetime,,,,,,,,,,,,,,,,,,,,,,
2023-07-05,13065.599892,7517.530231,7850.475106,5774.17654,2068.359091,10272.520138,375.0,2679.840895,1648.586214,375.0,375.0,1889.284042,5931.519476,2271.541276,789.353592,1849.57698,4260.397698,4694.712471,26311.526359,0.00000,100000.000000,73688.473641
2023-07-05,13069.764704,7520.902627,7837.092568,5774.17654,2068.717983,10279.109157,375.0,2679.840895,1648.586214,375.0,375.0,1889.284042,5931.519476,2271.541276,789.353592,1849.57698,4260.397698,4694.712471,26311.526359,0.19809,73689.576224,73689.576224
2023-07-06,12264.764704,7962.902627,7592.092568,5774.17654,1963.717983,9694.109157,375.0,2679.840895,1648.586214,375.0,375.0,1889.284042,5931.519476,2271.541276,789.353592,1849.57698,4260.397698,4694.712471,26311.526359,0.19809,72391.576224,72391.576224
2023-07-07,11862.264704,7631.402627,7382.092568,5774.17654,2068.717983,9649.109157,375.0,2679.840895,1648.586214,375.0,375.0,1889.284042,5931.519476,2271.541276,789.353592,1849.57698,4260.397698,4694.712471,26311.526359,0.19809,71507.576224,71507.576224
2023-07-10,11862.264704,6916.402627,6997.092568,5774.17654,1933.717983,9064.109157,375.0,2679.840895,1648.586214,375.0,375.0,1889.284042,5931.519476,2271.541276,789.353592,1849.57698,4260.397698,4694.712471,26311.526359,0.19809,69687.576224,69687.576224
